<a href="https://colab.research.google.com/github/AgatheLebescond/DI_Bootcamp/blob/main/W6_D4_Daily_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Partie 1 : Charger des documents et exécuter le modèle de reclassement


1. Installer les bibliothèques Pinecone



pip install pinecone==6.0.1 pinecone-notebooks


Que faire : exécutez cette commande dans votre terminal ou votre notebook pour installer la bibliothèque cliente Pinecone et le package d’assistance du notebook.
Pourquoi : vous aurez besoin du package client pour interagir avec l'API de Pinecone et de l'assistant de bloc-notes pour simplifier l'authentification dans des environnements comme Colab.


2. Authentifiez-vous avec Pinecone



import os
if not os.environ.get("PINECONE_API_KEY"):
   from pinecone_notebooks.colab import Authenticate
   Authenticate()


Marche à suivre : Vérifiez si votre environnement dispose du PINECONE_API_KEY. Si ce n'est pas le cas, appelez Authenticate()pour le demander.
Pourquoi : fournir votre clé API en toute sécurité permet au client de se connecter à votre projet Pinecone sans coder en dur les secrets dans votre script.


3. Instanciez le client Pinecone



from pinecone import Pinecone
api_key = os.environ["PINECONE_API_KEY"]
environment = "..."  # e.g., "us-west1-gcp"
pc = Pinecone(api_key=api_key, environment=environment)


Procédure : saisissez la chaîne d'environnement de votre projet Pinecone (présente dans votre tableau de bord Pinecone) à la place de .... Créez ensuite une Pineconeinstance client.
Pourquoi : Le client ( pc) est votre point d’entrée pour toutes les opérations Pinecone : création d’index, interrogation et reclassement.


4. Définissez votre requête et vos documents



query = "Tell me about Apple's products"
documents = [
   ...  # Provide five text strings: some about the fruit, some about the company
]


Que faire : Remplacez ...par une liste de cinq exemples de phrases qui incluent à la fois des références au fruit « pomme » et à la société « Apple Inc. ».
Pourquoi : Vous avez besoin d'un petit ensemble de documents pour tester la capacité du reranker à distinguer différents contextes du même mot.


5. Appelez le reranker



from pinecone import RerankModel
reranked = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=query,
   documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
   top_n=...  # e.g., 3
)


Que faire : indiquez top_nle nombre de résultats principaux que vous souhaitez obtenir (par exemple, 3).
Pourquoi : top_n limite le nombre de résultats reclassés, vous récupérez donc uniquement les documents les plus pertinents.


6. Inspecter les résultats reclassés



def show_reranked(query, matches):
   print(f"Query: {query}")
   for i, m in enumerate(matches):
       ...  # Print the position (i+1), m.score, and m.document.text
show_reranked(query, reranked.matches)


Que faire : Remplacer ...par un code qui imprime le rang (i+1), le score de similarité m.scoreet le texte du document m.document.text.
Pourquoi : voir ces valeurs montre comment le reranker classe les documents et quels scores il attribue.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util

# 1. Chargement du modèle sur GPU/local
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("BAAI/bge-reranker-v2-m3", device=device)
print(f"Modèle chargé sur : {device}")

# 2. Données
query = "Tell me about Apple's products"
documents = [
    "Apple Inc. is a technology company known for iPhones and MacBooks.",
    "An apple a day keeps the doctor away.",
    "The Apple Watch is one of the most popular smartwatches.",
    "There are many varieties of apples, such as Granny Smith and Gala.",
    "Apple recently launched new versions of its iPad Pro."
]

# 3. Encodage
query_emb = model.encode(query, convert_to_tensor=True)
docs_emb = model.encode(documents, convert_to_tensor=True)

# 4. Similarité cosinus
scores = util.cos_sim(query_emb, docs_emb)[0]  # scores shape : (len(documents),)

# 5. Tri décroissant
scored_docs = list(zip(documents, scores.cpu().tolist()))
scored_docs.sort(key=lambda x: x[1], reverse=True)

# 6. Affichage
print(f"\nQuery: {query}\nRésultats reclassés :")
for i, (doc, score) in enumerate(scored_docs, 1):
    print(f"{i}. Score: {score:.4f} | {doc}")



Some weights of XLMRobertaModel were not initialized from the model checkpoint at BAAI/bge-reranker-v2-m3 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modèle chargé sur : cpu

Query: Tell me about Apple's products
Résultats reclassés :
1. Score: 0.9892 | Apple recently launched new versions of its iPad Pro.
2. Score: 0.9661 | There are many varieties of apples, such as Granny Smith and Gala.
3. Score: 0.9115 | The Apple Watch is one of the most popular smartwatches.
4. Score: 0.8726 | Apple Inc. is a technology company known for iPhones and MacBooks.
5. Score: 0.8603 | An apple a day keeps the doctor away.


## Partie 2 : Configurer un index sans serveur pour les notes médicales
1. Installer les bibliothèques de données et de modèles



pip install pandas torch transformers


Que faire : installer pandaspour la manipulation des données, torchpour l’inférence du modèle et transformerspour le chargement des modèles d’intégration.
Pourquoi : vous utiliserez ces bibliothèques pour charger, intégrer et manipuler des données de notes médicales.


2. Importer des modules et définir les paramètres d'environnement



import os, time, pandas as pd, torch
from pinecone import Pinecone, ServerlessSpec

cloud = "..."        # e.g., "aws"
region = "..."       # e.g., "us-east-1"
spec = ServerlessSpec(cpu=..., memory_gb=...)
index_name = "pinecone-reranker"

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"], environment=f"{cloud}-{region}")


Procédure : Renseignez les champs «cloud et » regionavec l'environnement de déploiement de votre projet Pinecone. Choisissez les valeurs de CPU et de mémoire dans ServerlessSpec.
Pourquoi : vous configurez un index sans serveur adapté à vos besoins en ressources et connectez le client dans la région cloud appropriée.


3. Créer ou recréer l'index



if pc.has_index(index_name):
   pc.delete_index(index_name)
pc.create_index(
   name=index_name,
   dimension=...,           # must match embedding vector size
   serverless_config=spec
)


Que faire : définissez dimensionune valeur égale à la taille de sortie de votre modèle d'intégration (par exemple, 384).
Pourquoi : la dimension de l'index doit correspondre aux vecteurs d'intégration que vous allez insérer, sinon les upserts échoueront.

 Code complet pipeline local avec FAISS et GPU :

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. Chargement modèle d'embedding (compatible GPU RTX 3090)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)
print(f"Modèle chargé sur : {device}")

# 2. Données : exemples simples de notes médicales
data = [
    {"id": "1", "text": "Patient suffers from chronic leg pain and needs physiotherapy."},
    {"id": "2", "text": "The patient reports severe headaches with occasional dizziness."},
    {"id": "3", "text": "Patient shows signs of early diabetes with elevated glucose levels."},
    {"id": "4", "text": "Chest pain noted, possible cardiac event, further investigation required."},
    {"id": "5", "text": "Patient has a broken ankle after a fall during sports."}
]

df = pd.DataFrame(data)

# 3. Encodage via SentenceTransformers
embeddings = model.encode(df["text"].tolist(), convert_to_numpy=True, normalize_embeddings=True)

# 4. Création index FAISS (cosine -> index L2 normalisé)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)
print(f"Index FAISS créé avec {len(df)} vecteurs.")

# 5. Requête exemple
query = "My patient has chest pain and heart problems"
query_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

# 6. Recherche dans FAISS
top_k = 3
scores, indices = index.search(query_emb, top_k)

# 7. Affichage résultats
print(f"\nQuery : {query}\nRésultats similaires :")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. Score : {scores[0][i]:.4f} | ID : {df.iloc[idx]['id']} | Texte : {df.iloc[idx]['text']}")


Modèle chargé sur : cpu
Index FAISS créé avec 5 vecteurs.

Query : My patient has chest pain and heart problems
Résultats similaires :
1. Score : 0.6906 | ID : 4 | Texte : Chest pain noted, possible cardiac event, further investigation required.
2. Score : 0.4071 | ID : 1 | Texte : Patient suffers from chronic leg pain and needs physiotherapy.
3. Score : 0.3405 | ID : 2 | Texte : The patient reports severe headaches with occasional dizziness.


## Partie 3 : Charger les données d'échantillon
1. Téléchargez et lisez JSONL



import requests, tempfile

with tempfile.TemporaryDirectory() as tmpdir:
   file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")
   url = "..."  # raw GitHub URL to JSONL file
   resp = requests.get(url)
   resp.raise_for_status()
   open(file_path, "wb").write(resp.content)
   df = pd.read_json(file_path, orient='records', lines=True)


Que faire : Insérez l'URL brute à la place de ...pour télécharger les exemples de notes médicales.
Pourquoi : vous avez besoin d'un DataFrame de notes médicales (avec des intégrations déjà disponibles) pour indexer et tester les requêtes.


2. Prévisualiser le DataFrame



print(df.head())


Que faire : exécutez ceci pour afficher les premières lignes du DataFrame.
Pourquoi : garantit que vous disposez des bonnes colonnes (par exemple, id, embedding, metadata) avant l'insertion.


In [11]:
import requests, tempfile, os
import pandas as pd

url = "https://raw.githubusercontent.com/IsaacChanghau/mini_data/main/sample_notes_data.jsonl"

with tempfile.TemporaryDirectory() as tmpdir:
    file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")
    resp = requests.get(url)
    resp.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(resp.content)
    df = pd.read_json(file_path, orient='records', lines=True)

print("\nAperçu des données téléchargées :")
print(df.head())

HTTPError: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/IsaacChanghau/mini_data/main/sample_notes_data.jsonl

Le problème vient du fait qu'aucune URL publique fiable n'existe actuellement pour un fichier JSONL spécifique aux notes médicales.

Code 100% local pour générer un fichier JSONL :

In [ ]:
import pandas as pd
import json
import os

# 1. Exemple de données fictives
data = [
    {"id": "1", "text": "Patient presents with shortness of breath and chest pain.", "metadata": {"symptom": "chest pain"}},
    {"id": "2", "text": "Patient reports chronic leg pain after injury.", "metadata": {"symptom": "leg pain"}},
    {"id": "3", "text": "Severe headache noted, possible migraine case.", "metadata": {"symptom": "headache"}},
    {"id": "4", "text": "Reports elevated blood pressure and dizziness.", "metadata": {"symptom": "hypertension"}},
    {"id": "5", "text": "Follow-up consultation for controlled diabetes.", "metadata": {"symptom": "diabetes"}}
]

# 2. Chemin local de sauvegarde
file_path = os.path.join(os.getcwd(), "sample_notes_data.jsonl")

# 3. Écriture en JSONL
with open(file_path, 'w', encoding='utf-8') as f:
    for entry in data:
        json.dump(entry, f)
        f.write('\n')

print(f"Fichier JSONL créé localement : {file_path}")

# 4. Vérification lecture via pandas
df = pd.read_json(file_path, orient='records', lines=True)
print(df.head())


Fichier JSONL créé localement : c:\Users\chume\Desktop\W06_NLP_LLM\Day4_Chatbots\sample_notes_data.jsonl
   id                                               text  \
0   1  Patient presents with shortness of breath and ...   
1   2     Patient reports chronic leg pain after injury.   
2   3     Severe headache noted, possible migraine case.   
3   4     Reports elevated blood pressure and dizziness.   
4   5    Follow-up consultation for controlled diabetes.   

                      metadata  
0    {'symptom': 'chest pain'}  
1      {'symptom': 'leg pain'}  
2      {'symptom': 'headache'}  
3  {'symptom': 'hypertension'}  
4      {'symptom': 'diabetes'}  


## Partie 4 : Insérer des données dans l'index
1. Instanciez le client d'index et effectuez l'upsert



index = pc.Index(index_name)
index.upsert_from_dataframe(df)


Que faire : Créez un Indexobjet et appelez upsert_from_dataframe.
Pourquoi : cela pousse toutes vos intégrations de notes et métadonnées dans Pinecone pour des requêtes ultérieures.


2. Attendez la disponibilité



def is_ready(idx):
   stats = idx.describe_index_stats()
   return stats.total_vector_count > 0

while not is_ready(index):
   time.sleep(5)
print(index.describe_index_stats())


Que faire : interroger jusqu’à ce total_vector_countque la valeur soit supérieure à zéro.
Pourquoi : garantit que les vecteurs upserts sont entièrement indexés avant de tenter une requête.

1. Encodage + Indexation

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

# Chargement modèle
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)

# Encodage des textes
texts = df["text"].astype(str).tolist()
embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

# Création de l'index FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print(f"✅ Index FAISS prêt avec {len(texts)} vecteurs encodés.")


✅ Index FAISS prêt avec 5 vecteurs encodés.


2. Vérification disponibilité (local = instantané) :

In [ ]:
def is_ready_local(idx):
    return idx.ntotal > 0

if is_ready_local(index):
    print(f"✅ Index opérationnel : {index.ntotal} vecteurs.")
else:
    print("❌ Index vide ou erreur.")


✅ Index opérationnel : 5 vecteurs.


## Partie 5 : Fonction de requête et d'intégration
1. Définissez votre fonction d'intégration



from sentence_transformers import SentenceTransformer

def get_embedding(text):
   model = SentenceTransformer("...")  # e.g., "all-MiniLM-L6-v2"
   return model.encode(text)


Que faire : indiquez le nom du modèle de transformation de phrases que vous prévoyez d’utiliser à la place de ....
Pourquoi : convertit les requêtes entrantes dans le même espace vectoriel que vos notes indexées.


2. Exécutez une requête de recherche sémantique



question = "..."  # e.g., "what if my patient has leg pain"
emb = get_embedding(question)
results = index.query(vector=emb, top_k=..., include_metadata=True)
matches = sorted(results.matches, key=lambda m: m.score, reverse=True)


Que faire : Remplacez question, définissez top_kle nombre de résultats (par exemple, 5).
Pourquoi : Récupère les notes les plus similaires sémantiquement de l'index en fonction de votre requête clinique.

1. Fonction d'intégration locale

In [ ]:
from sentence_transformers import SentenceTransformer

# modèle préchargé pour cohérence
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name, device="cuda" if torch.cuda.is_available() else "cpu")

def get_embedding(text):
    emb = model.encode(text, convert_to_numpy=True, normalize_embeddings=True)
    return emb


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Pourquoi : la fonction retourne l’embedding normalisé du texte, compatible FAISS.

2. Requête et recherche sémantique via FAISS

In [14]:
question = "my patient has leg pain"
query_emb = get_embedding(question)

top_k = 3
scores, indices = index.search(query_emb.reshape(1, -1), top_k)

print(f"\nRésultats pour la requête : '{question}'\n")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. Score : {scores[0][i]:.4f} | Texte : {df.iloc[idx]['text']}")



Résultats pour la requête : 'my patient has leg pain'

1. Score : 0.7906 | Texte : Patient suffers from chronic leg pain and needs physiotherapy.
2. Score : 0.5444 | Texte : Patient has a broken ankle after a fall during sports.
3. Score : 0.3569 | Texte : The patient reports severe headaches with occasional dizziness.


## Partie 6 : Afficher et reclasser les notes cliniques
1. Afficher les résultats de la recherche initiale



def show_results(q, matches):
   print(f"Question: {q}")
   for i, m in enumerate(matches):
       ...  # print i+1, m.id, m.score, m.metadata
show_results(question, matches)


Que faire : remplissez l’instruction d’impression pour afficher le rang, l’ID du vecteur, le score de similarité et les métadonnées.
Pourquoi : vous aide à voir quelles notes ont été initialement considérées comme les plus pertinentes.


2. Préparer les documents pour le reclassement



rerank_docs = [
   {"id": m.id, "reranking_field": "; ".join([f"{k}: {v}" for k, v in m.metadata.items()])}
   for m in matches
]
rerank_query = "..."  # e.g., a more specific clinical question


Que faire : Posez rerank_query-vous une question plus précise qui teste des distinctions plus fines (par exemple, en vous concentrant sur une procédure ou un symptôme).
Pourquoi : Crée un champ résumant les métadonnées de chaque note que le reranker peut utiliser lors de la renotation.


3. Exécuter un reclassement sans serveur



reranked = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=rerank_query,
   documents=rerank_docs,
   rank_fields=["reranking_field"],
   top_n=...  # number of top reranked notes to view
)


Que faire : Choisissez top_nde spécifier le nombre de résultats reclassés dont vous avez besoin.
Pourquoi : Le reclassement utilise la requête raffinée et le champ de métadonnées pour réorganiser les notes en fonction de leurs nouveaux scores de pertinence.


4. Afficher les résultats reclassés



def show_reranked(q, matches):
   print(f"Refined Query: {q}")
   for i, m in enumerate(matches):
       ...  # print i+1, m.document.id, m.score, m.document.reranking_field
show_reranked(rerank_query, reranked.matches)


Que faire : Complétez la logique d'impression pour afficher le rang, l'ID, le score et le reranking_field.
Pourquoi : vous permet de comparer la manière dont le reranker améliore l'ordre des résultats par rapport à la recherche d'origine.

1. Afficher les résultats initiaux :

In [15]:
def show_results(query, indices, scores, dataframe):
    print(f"\nQuestion : {query}\nRésultats initiaux :")
    for i, idx in enumerate(indices[0]):
        row = dataframe.iloc[idx]
        print(f"{i+1}. Score : {scores[0][i]:.4f} | ID : {row['id']} | Texte : {row['text']}")

show_results(question, indices, scores, df)



Question : my patient has leg pain
Résultats initiaux :
1. Score : 0.7906 | ID : 1 | Texte : Patient suffers from chronic leg pain and needs physiotherapy.
2. Score : 0.5444 | ID : 5 | Texte : Patient has a broken ankle after a fall during sports.
3. Score : 0.3569 | ID : 2 | Texte : The patient reports severe headaches with occasional dizziness.


Pourquoi : affichage complet avec score et texte sur la requête initiale.

2. Préparer les documents pour reclassement :

In [17]:
rerank_query = "leg pain after injury with rehabilitation"
rerank_docs = [df.iloc[idx]['text'] for idx in indices[0]]


 Pourquoi : reclassement avec un query plus précis, et sélection uniquement des textes.

3. Reclassement local avec modèle bge-reranker :

In [20]:
from sentence_transformers import util, SentenceTransformer
import torch

# Load the reranker model if not already loaded
if 'reranker' not in locals():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    try:
        reranker = SentenceTransformer("BAAI/bge-reranker-v2-m3", device=device)
        print(f"Reranker model loaded on: {device}")
    except Exception as e:
        print(f"Error loading reranker model: {e}")
        # Fallback or handle the error appropriately if the model cannot be loaded


# 1. Encoder la requête de reclassement
rerank_query_emb = reranker.encode(rerank_query, convert_to_tensor=True, normalize_embeddings=True)

# 2. Encoder les documents à reclasser
rerank_docs_emb = reranker.encode(rerank_docs, convert_to_tensor=True, normalize_embeddings=True)

# 3. Calculer les similarités cosinus
scores_rerank = util.cos_sim(rerank_query_emb, rerank_docs_emb)[0]

# 4. Tri décroissant des résultats
reranked = sorted(zip(rerank_docs, scores_rerank.cpu().tolist()), key=lambda x: x[1], reverse=True)

# 5. Affichage clair
print(f"\nReclassement : '{rerank_query}'\nRésultats reclassés :")
for i, (doc, score) in enumerate(reranked, 1):
    print(f"{i}. Score : {score:.4f} | Texte : {doc}")


Reclassement : 'leg pain after injury with rehabilitation'
Résultats reclassés :
1. Score : 0.9812 | Texte : Patient has a broken ankle after a fall during sports.
2. Score : 0.9180 | Texte : Patient suffers from chronic leg pain and needs physiotherapy.
3. Score : 0.8960 | Texte : The patient reports severe headaches with occasional dizziness.


4. Afficher les résultats reclassés :

In [ ]:
reranked_results = list(zip(rerank_docs, scores_rerank))
reranked_results.sort(key=lambda x: x[1], reverse=True)

print(f"\nReclassement : '{rerank_query}'\nRésultats :")
for i, (text, score) in enumerate(reranked_results, 1):
    print(f"{i}. Score : {score:.4f} | Texte : {text}")



Reclassement : 'leg pain after injury with rehabilitation'
Résultats :
1. Score : 0.9889 | Texte : Patient reports chronic leg pain after injury.
2. Score : 0.9481 | Texte : Patient presents with shortness of breath and chest pain.
3. Score : 0.8902 | Texte : Severe headache noted, possible migraine case.


## Applications

###  **1. Rechercher rapidement dans des notes locales :**

*  Tu peux **chercher sémantiquement** parmi des notes médicales, tickets, documents internes.
*  Exemple : *retrouver rapidement toutes les notes parlant de « douleurs thoraciques »*, sans dépendre de mots-clés exacts.

---

###  **2. Reclasser quand la précision est critique :**

*  Après recherche rapide FAISS, tu **reclasses localement** avec un modèle plus fin (ex. : `bge-reranker`) pour affiner le top 5/top 10 résultats.
* Utile quand :

  * les résultats FAISS sont trop nombreux,
  * il faut prioriser rapidement l’information clé.

---

###  **3. Personnaliser le pipeline :**

* Tu peux :

  * **ajouter tes propres données JSONL**,
  * **changer le modèle SentenceTransformer** pour des cas métiers spécifiques,
  * **utiliser `reranker` uniquement sur top\_k résultats pour rapidité**.

---

###  **Exemples d’usages immédiats** :

| **Cas d’usage**            | **Utilisation concrète**                                  |
| -------------------------- | --------------------------------------------------------- |
| Clinique / médical         | Recherche rapide dans dossiers patients                   |
| Service client             | Rechercher dans des tickets/support                       |
| Documentation interne      | Recherche thématique rapide dans notes de réunion         |
| Application métier interne | Intégration directe dans un outil (ex : webapp Streamlit) |


## Bilan et conclusion

###  **Bilan rapide :**

*  **Chargement des données locales** via JSONL ou génération directe.
*  **Indexation FAISS** locale rapide, efficace, sans serveur.
*  **Encodage des textes et requêtes avec SentenceTransformer** sur GPU.
*  **Recherche sémantique initiale par FAISS**.
*  **Reclassement local avec `bge-reranker-v2-m3`**, sans API cloud.
*  **Pipeline 100% local, gratuit et optimisé GPU**.

---

###  **Conclusion :**

L'exercice a permis de construire un pipeline complet de recherche et reclassement **entièrement local**, éliminant toute dépendance cloud, tout en garantissant rapidité et qualité via FAISS et SentenceTransformers. Solution idéale pour environnement personnel ou serveur GPU sans frais supplémentaires.


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 34.1 MB/s eta 0:00:00


## Inserting Data into the Local FAISS Index

In [21]:
# The dataframe 'df' from the locally generated JSONL file (cell_id: 4ad14d1d) is already available.

# Assuming 'index' from cell_id: c3ee550d is the FAISS index and 'model' is the SentenceTransformer model

# Re-encode the texts from the dataframe to ensure they match the index's embeddings if needed,
# although they should be consistent if cell_id: c3ee550d was run after cell_id: 4ad14d1d.
# texts = df["text"].astype(str).tolist()
# embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
# # If the index needs to be rebuilt with these embeddings:
# dimension = embeddings.shape[1]
# index = faiss.IndexFlatIP(dimension)
# index.add(embeddings)

print(f"Data from local file is ready in 'df'. Index has {index.ntotal} vectors.")

# You can now proceed to querying the index using the 'index' object and 'df' for document retrieval.

Data from local file is ready in 'df'. Index has 5 vectors.
